In [1]:
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__)))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd()))

SRC_PATH = os.path.join(TOOLS_PATH)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src


In [2]:
import os
import openai
from langsmith import Client

LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT= "https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY") 
openai.api_key=os.environ["OPENAI_API_KEY"]

# Set tracing in LangGraph
from langchain_core.tracers import ConsoleCallbackHandler
callbacks = [ConsoleCallbackHandler()]  #

In [1]:
# src/main.py

from graph.workflow import build_graph

def run_once(user_query: str):
    """
    Build the graph, run it end‑to‑end, and return the final AgentState.
    """
    # 1) Compile the workflow
    graph = build_graph()

    # 2) Prepare the initial state for LangGraph
    #    All nodes share this same AgentState schema.
    initial_state = {
        "messages": [ {"role": "user", "content": user_query} ]
    }

    # 3) Invoke the graph synchronously (follows supervisor→enhancer→search→END)
    final_state = graph.invoke(initial_state)

    # 4) Inspect or return the final state
    return final_state

if __name__ == "__main__":
    query = "Find 5 B2B SaaS startups in India "
    result = run_once(query)

    print("=== Final State ===")
    print(f"Enhanced Query   : {result.get('enhanced_query')}")
    print(f"Filters          : {result.get('filters')}")
    print(f"Top‑K (k)        : {result.get('k')}")
    print(f"Results          : {result.get('retrieved_results')}")
    print(f"Trace Actions    : {result.get('actions')}")
    print(f"Trace Observations: {result.get('observations')}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src
🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
📌 Collection Name: indian_startups

[DEBUG] Enhancer Node received user_input: Find 5 B2B SaaS startups in India 
[DEBUG] Enhancer payload: {'input': 'Find 5 B2B SaaS startups in India ', 'intermediate_steps': []}
[DEBUG] AgentAction: Thought: what to do next
Action: keyword_extractor
Action Input: Find 5 B2B SaaS startups in India 
[DEBUG] raw final_output: None
[DEBUG] Parsed enhanced_query: 
[DEBUG] Parsed filters: {}
[DEBUG] Parsed k: None

[DEBUG] Enhancer Node received user_input: Find 5

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT